# <h1 align="center"><font color="red">Como aproveitar Function Calling da OpenAI para recuperação de dados financeiros?</font></h1>

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>


Notebook baseado no maravilhoso tutorial de [Hanane DUPOUY]()

Procurando recuperar informações financeiras da empresa de forma eficiente? Aqui está uma rápida análise de como você pode usar a chamada de função do `OpenAI`:

1. `Defina suas funções:`

Comece integrando APIs externas e especificando os dados exatos que você deseja extrair delas.

2. `Configure esquemas de função:`

Para cada função, forneça detalhes essenciais, como o nome da função, descrição e os parâmetros necessários (por exemplo, propriedades, tipos).

3. `Invoque o modelo:` usei `GPT-4o-mini`

Use o método `completion` de bate-papo, passando os parâmetros da função e defina `function_call=auto` para deixar o modelo determinar quando chamar as funções.

Todo o processo é descrito claramente no notebook para fácil implementação!


Os Dados:

* [FMP: Financial data for every need](https://site.financialmodelingprep.com/)

In [8]:
#%pip install openai -q
# pip install requests
# pip install python-dotenv

#from google.colab import userdata
#OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
#FINANCIAL_MODELING_PREP_API_KEY = userdata.get('FINANCIAL_MODELING_PREP_API_KEY')

import requests
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
#openai.api_key  = os.environ['OPENAI_API_KEY']
Eddy_key_openai  = os.environ['OPENAI_API_KEY']
FINANCIAL_MODELING_PREP_API_KEY = os.environ['FINANCIAL_MODELING_PREP_API_KEY'] # --> https://site.financialmodelingprep.com/developer/docs/dashboard


from openai import OpenAI
client = OpenAI(api_key=Eddy_key_openai)


# <font color="gree">Definimos nossas funções</font>

In [5]:
def get_stock_price(symbol):
    """
    Fetch the current stock price for the given symbol, the current volume, the average price 50d and 200d, EPS, PE and the next earnings Announcement.
    """
    url = f"https://financialmodelingprep.com/api/v3/quote-order/{symbol}?apikey={FINANCIAL_MODELING_PREP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    try:
        price = data[0]['price']
        volume = data[0]['volume']
        priceAvg50 = data[0]['priceAvg50']
        priceAvg200 = data[0]['priceAvg200']
        eps = data[0]['eps']
        pe = data[0]['pe']
        earningsAnnouncement = data[0]['earningsAnnouncement']
        return {"symbol": symbol.upper(), "price": price, "volume":volume,"priceAvg50":priceAvg50, "priceAvg200":priceAvg200, "EPS":eps, "PE":pe, "earningsAnnouncement":earningsAnnouncement }
    except (IndexError, KeyError):
        return {"error": f"Could not fetch price for symbol: {symbol}"}

def get_company_financials(symbol):
    """
    Fetch basic financial information for the given company symbol such as the industry, the sector, the name of the company, and the market capitalization.
    """
    url = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={FINANCIAL_MODELING_PREP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    try:
        results = data[0]
        financials = {
            "symbol": results["symbol"],
            "companyName": results["companyName"],
            "marketCap": results["mktCap"],
            "industry": results["industry"],
            "sector": results["sector"],
            "website": results["website"],
            "beta":results["beta"],
            "price":results["price"],
        }
        return financials
    except (IndexError, KeyError):
        return {"error": f"Could not fetch financials for symbol: {symbol}"}

def get_income_statement(symbol):
    """
    Fetch last income statement for the given company symbol such as revenue, gross profit, net income, EBITDA, EPS.
    """
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{symbol}?period=annual&apikey={FINANCIAL_MODELING_PREP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    try:
        results = data[0]
        financials = {
            "date": results["date"],
            "revenue": results["revenue"],
            "gross profit": results["grossProfit"],
            "net Income": results["netIncome"],
            "ebitda": results["ebitda"],
            "EPS": results["eps"],
            "EPS diluted":results["epsdiluted"]
        }
        return data, financials
    except (IndexError, KeyError):
        return {"error": f"Could not fetch financials for symbol: {symbol}"}
    

# <font color="gree">Configuramos o Schema da Função</font>

In [6]:
# Define the function schemas for OpenAI
functions = [
    {
        "name": "get_stock_price",
        "description": "Get the current stock price for a given company symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., AAPL for Apple."
                }
            },
            "required": ["symbol"],
        },
    },
    {
        "name": "get_company_financials",
        "description": "Get basic financial information for a given company.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., GOOGL for Alphabet Inc."
                }
            },
            "required": ["symbol"],
        },
    },
    {
        "name": "get_income_statement",
        "description": "Get the last income statement for a given company symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., AMZN for Amazon."
                }
            },
            "required": ["symbol"],
        },
    },
]

# <font color="gree">Invoke the Model: GPT-4o-mini and CHAT</font>

In [9]:
# Chat Function:
MODEL_NAME = "gpt-4o-mini"

def chat_with_LLM(model_name = MODEL_NAME):
    messages = []
    print("Sou seu Assistente de Analista Financeiro! Pergunte-me qualquer coisa sobre preços de ações, finanças de empresas ou declaração de renda.")
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Assistant: A discussão foi encerrada!")
            break

        messages.append({"role": "user", "content": user_input})

        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
            functions=functions,
            function_call="auto",  # Let the model decide if a function needs to be called
        )

        response_message = response.choices[0].message

        # Check if the assistant wants to call a function
        if response_message.function_call:
            function_name = response_message.function_call.name
            arguments = response_message.function_call.arguments

            # Parse the arguments
            import json
            args = json.loads(arguments)

            # Call the appropriate function
            if function_name == "get_stock_price":
                function_response = get_stock_price(**args)
            elif function_name == "get_company_financials":
                function_response = get_company_financials(**args)
            elif function_name == "get_income_statement":
                function_response = get_income_statement(**args)
            else:
                function_response = {"error": "Function not found."}

            # Add the function's response to the conversation
            messages.append(response_message)  # Assistant's function call
            messages.append({
                "role": "function",
                "name": function_name,
                "content": json.dumps(function_response),
            })

            # Get the assistant's final response
            second_response = client.chat.completions.create(
                model=MODEL_NAME,
                messages=messages,
            )

            final_message = second_response.choices[0].message.content
            print(f"Assistant: {final_message}")
            messages.append({"role": "assistant", "content": final_message})
        else:
            # If no function call is needed, just output the assistant's message
            assistant_reply = response_message.content
            print(f"Assistant: {assistant_reply}")
            messages.append({"role": "assistant", "content": assistant_reply})

# Run the chat function
if __name__ == "__main__":
    chat_with_LLM()


Sou seu Assistente de Analista Financeiro! Pergunte-me qualquer coisa sobre preços de ações, finanças de empresas ou declaração de renda.
Assistant: O último preço das ações da Amazon (AMZN) é de **191,60 USD**.
Assistant: O último preço das ações da Nike (NKE) é de **86,52 USD**.
Assistant: A receita de vendas da NVIDIA (NVDA) para o último ano fiscal encerrado em 28 de janeiro de 2024 foi de **60.922.000.000 USD** (60,9 bilhões de dólares).
Assistant: Parece que sua mensagem não foi enviada. Como posso ajudar você?
Assistant: Desde a última atualização do meu treinamento, a OpenAI não divulgou publicamente informações financeiras específicas, como receita de vendas, pois é uma empresa privada. Para informações precisas e atualizações, pode ser necessário consultar fontes financeiras ou comunicados oficiais da OpenAI. Se precisar de informações sobre outra empresa ou assunto, estou à disposição para ajudar!
Assistant: A última receita reportada pela Apple (AAPL) para o ano fiscal ence